In [2]:
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
import pickle
from sklearn.utils import shuffle

from IPython.display import clear_output, display
import os.path
from os import path
import shutil
from pathlib import Path
from glob import glob
from collections import Counter
import builtins

In [4]:
pgnfile= "C:/mestrado/datasets/unified/vocabulary/vocabulary/from-handmade/pgn_10k_eng.txt"
with open(pgnfile) as f:
    lines = [line.rstrip() for line in f]
    
lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
lines= [ln[8:] for ln in lines]
lines= [ln.split() for ln in lines]

In [5]:
len( lines)

10000

In [6]:
lines[0]

['d4',
 'f5',
 'Nf3',
 'g6',
 'g3',
 'Bg7',
 'Bg2',
 'Nf6',
 'b4',
 'c6',
 'c4',
 'd5',
 'Qb3',
 'Be6',
 'c5',
 'Ne4',
 'O-O',
 'Nd7',
 'Bf4',
 'O-O',
 'Nbd2',
 'Bf7',
 'Qe3',
 'Qe8',
 'Bh6',
 'Ndf6',
 'Ne5',
 'Nxd2',
 'Qxd2',
 'Nd7',
 'Bxg7',
 'Kxg7',
 'Qf4',
 'Be6',
 'a4',
 'Nxe5',
 'Qxe5+',
 'Rf6',
 'f4',
 'Kg8',
 'Rfb1',
 'Bc8',
 'Qc7',
 'Qd7',
 'Qxd7',
 'Bxd7',
 'b5',
 'Rff8',
 'e3',
 'e6',
 'Bf1',
 'Kf7',
 'a5',
 'Rfb8',
 'Rb2',
 'Ke7',
 'Rab1',
 'Rc8',
 'bxc6',
 'bxc6',
 'Ba6',
 'Rc7',
 'Rb8',
 'Rxb8',
 'Rxb8',
 'Be8',
 'Bb7',
 'Bd7',
 'a6',
 'Be8',
 'Ra8',
 'Bd7',
 'Kf2',
 'Kf6',
 'Rh8',
 'Kg7',
 'Ra8',
 'Kf6',
 'Ke2',
 'h6',
 'Kd2',
 'g5',
 'Kc3',
 'gxf4',
 'exf4']

In [7]:
flat_tokens16 = [t for _list in lines for t in _list[0:16]]

In [9]:
print( 'len( flat)', len( flat_tokens16))

len( flat) 157964


In [10]:
count16= Counter( flat_tokens16)
print( 'len( count16)', len( count16))

len( count16) 620


In [14]:
counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
counted16_k= [k for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)]
print( len( counted16_k))
print( counted16_k)

620
['O-O', 'Nf3', 'Nf6', 'd4', 'e4', 'Nc3', 'd5', 'e6', 'Nc6', 'c5', 'd6', 'c4', 'e5', 'g6', 'Bg7', 'cxd4', 'Be7', 'c6', 'g3', 'Bg2', 'a6', 'e3', 'Nxd4', 'Bd3', 'c3', 'Bg5', 'Be2', 'exd5', 'd3', 'Nbd7', 'Be3', 'cxd5', 'Bc4', 'b6', 'b5', 'Bg4', 'f4', 'h6', 'Bb4', 'h3', 'Bb7', 'Bb5', 'a3', 'Nbd2', 'b3', 'Bd6', 'Bd7', 'Bf5', 'Bf4', 'Qc7', 'Nxe4', 'f5', 'exd4', 'Nd7', 'dxc4', 'Qd2', 'dxe5', 'Re1', 'Qc2', 'Nxd5', 'Bc5', 'f3', 'Nd2', 'a4', 'Bb2', 'dxe4', 'Bd2', 'Nge2', 'Qb6', 'Bb3', 'Ne7', 'Nd5', 'Nge7', 'Ne4', 'Be6', 'Qe2', 'bxc3', 'Ne2', 'Qxd5', 'Qa5', 'dxc5', 'Ba4', 'Ne5', 'Nxe5', 'a5', 'Bxf6', 'b4', 'Nxc3', 'Bxc6', 'Nb6', 'Qb3', 'Nfd7', 'Nb3', 'Bxc4', 'bxc6', 'Bxc3+', 'Bb4+', 'Bxf3', 'Qe7', 'Qxd4', 'h4', 'Bh4', 'Na6', 'Bb5+', 'Re8', 'f6', 'h5', 'Nxc6', 'Na3', 'Bxc3', 'Ngf3', 'Ngf6', 'O-O-O', 'g4', 'Ng4', 'dxc6', 'Nd4', 'Ng3', 'Bg6', 'Bxc5', 'Rb1', 'Nb5', 'g5', 'Nh6', 'Bb6', 'Ndb5', 'Bh5', 'Rb8', 'Ba6', 'Qf3', 'Na5', 'Ng5', 'Qd7', 'cxb5', 'Qg4', 'Qd3', 'dxc3', 'exd6', 'exf6', 'Nh5', 'Bxd

In [18]:
def count_16( pgnfile):
    with open(pgnfile) as f:
        lines = [line.rstrip() for line in f]

    lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
    lines= [ln[8:] for ln in lines]
    lines= [ln.split() for ln in lines]

    flat_tokens16 = [t for _list in lines for t in _list[0:16]]

    count16= Counter( flat_tokens16)

    counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
    counted16_k= [k for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)]
    print( len( counted16_k))
    print( counted16_k)
    
count_16( "C:/mestrado/datasets/unified/vocabulary/vocabulary/from-handmade/pgn_10k_eng.txt")


424
['Nf6', 'Nf3', 'd4', 'e4', 'Nc3', 'd5', 'e6', 'c5', 'c4', 'O-O', 'd6', 'Nc6', 'e5', 'g6', 'Bg7', 'g3', 'cxd4', 'c6', 'Bg2', 'e3', 'Nxd4', 'Be7', 'a6', 'c3', 'Bg5', 'exd5', 'Be2', 'd3', 'Bd3', 'Bc4', 'cxd5', 'Nbd7', 'Bb5', 'b6', 'Bb4', 'Be3', 'f4', 'Bg4', 'Bb7', 'b5', 'Bf5', 'f5', 'b3', 'h6', 'a3', 'Bf4', 'Nxe4', 'dxc4', 'Bc5', 'Bd6', 'exd4', 'Nbd2', 'Nd2', 'h3', 'Bd7', 'Nxd5', 'dxe4', 'Nd7', 'Bb2', 'Qc7', 'Qxd5', 'Nd5', 'Ba4', 'f3', 'Qc2', 'Qb6', 'dxe5', 'Bd2', 'Nge2', 'Re1', 'Bb4+', 'Ne7', 'Ne4', 'Nge7', 'Nfd7', 'a4', 'bxc3', 'Bxc3+', 'Bxc6', 'Nxe5', 'Nxc3', 'Qa5', 'Bxf6', 'Bb5+', 'Qxd4', 'Nb6', 'Ne2', 'Qe2', 'Bb3', 'bxc6', 'Ngf3', 'dxc5', 'Qb3', 'h4', 'Bh4', 'Qd2', 'b4', 'dxc6', 'Qe7', 'Ndb5', 'Ngf6', 'Nxc6', 'Be6', 'Ng3', 'Ne5', 'h5', 'Bg6', 'Bxc4', 'Bxf3', 'f6', 'cxb5', 'Nb5', 'Nb3', 'Nh6', 'Na6', 'Nd4', 'g4', 'dxc3', 'a5', 'Bb6', 'Ng5', 'Bxc3', 'exf6', 'exf4', 'exd6', 'fxe4', 'Ba6', 'Qf6', 'g5', 'Qa4+', 'Ng4', 'gxf6', 'Qa5+', 'Nxf6+', 'Bh5', 'Bxc5', 'Qxf6', 'Na5', 'Bxd3', 'Bxd

In [20]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

In [54]:
files = glob( 'C:/mestrado/datasets/final/datasets_pool/hand/labels/*.pgn')
lines = []
for f in files:
    lines.append( read_label(f))
# files = glob( 'C:/mestrado/datasets/final/datasets_pool/torneio-except-test/labels/*.pgn')
files = glob( 'C:/mestrado/datasets/final/test-set/new-random-8lines--v1--test-only/new-random-8lines--v1--test-only/test/labels/*.pgn')
for f in files:
    lines.append( read_label(f))
len( lines)

2124

In [55]:
_lines= [ln.split() for ln in lines]

flat_tokens16 = [t for _list in _lines for t in _list[0:16]]

count16= Counter( flat_tokens16)

counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
counted16_k= [k for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)]
print( len( counted16_k))
print( counted16_k)

175
['O-O', 'Nf6', 'Nf3', 'd4', 'Nc3', 'e4', 'e6', 'c5', 'd5', 'Nc6', 'c4', 'd6', 'e5', 'g6', 'Bg7', 'cxd4', 'Be7', 'g3', 'Bg2', 'a6', 'Nxd4', 'e3', 'c6', 'Bd3', 'Be2', 'Bg5', 'Nbd7', 'd3', 'c3', 'exd5', 'Be3', 'cxd5', 'b6', 'Bc4', 'Bb7', 'Nbd2', 'h6', 'b5', 'b3', 'Bb4', 'Qc7', 'f4', 'Bb5', 'h3', 'Re1', 'a3', 'Qd2', 'Bb3', 'Bb2', 'Qc2', 'exd4', 'Bd6', 'Bg4', 'Bd7', 'dxc4', 'Bf4', 'a4', 'Nge2', 'f3', 'Nge7', 'Qb6', 'Nb3', 'Nd2', 'Bf5', 'Ba4', 'Be6', 'Nd7', 'dxc5', 'Bc5', 'Nxd5', 'f5', 'a5', 'Nxe4', 'Bxf6', 'dxe4', 'Bh4', 'Bxc4', 'Ne7', 'Ne2', 'Nfd7', 'Re8', 'Nxc6', 'bxc6', 'Qxd4', 'Qe2', 'Ne5', 'Qb3', 'Qa5', 'Na6', 'Bxc6', 'b4', 'Bd2', 'Nb6', 'bxc3', 'Qxd5', 'Bxc5', 'Ne4', 'Bxc3+', 'Ngf3', 'Qe7', 'Nd5', 'Rb8', 'Bb4+', 'Rb1', 'Bb5+', 'dxc6', 'O-O-O', 'h5', 'Nd4', 'Na5', 'Bxc3', 'Nxe5', 'Ngf6', 'exf6', 'f6', 'g5', 'Qd3', 'gxf6', 'Qf3', 'Qd6', 'Nh5', 'Qxf6', 'Bg3', 'Nb5', 'Qa4', 'Nc5', 'Qd7', 'Bxd6', 'Ng5', 'h4', 'Qe1', 'Qd1', 'Ndb5', 'Nxf6+', 'Nb4', 'Qe8', 'Ba7', 'Nce7', 'Qf6', 'Qxd6', 'Q

In [123]:
def read_files( paths):
    files = []
    for path in paths:
        print ( path)
        path_files = glob( path)
        for f in path_files:
            files.append( f)
    return files

files = read_files( [
    'C:/mestrado/datasets/final/datasets_pool/hand/labels/*.pgn',
    'C:/mestrado/datasets/final/datasets_pool/torneio-except-test/labels/*.pgn',
    'C:/mestrado/datasets/final/test-set/new-random-8lines--v1--test-only/new-random-8lines--v1--test-only/test/labels/*.pgn'
])
print( len( files))
print( files[0:2])
lines = []
for f in files:
    lines.append( read_label(f))

_lines= [ln.split() for ln in lines]

flat_tokens16 = [t for _list in _lines for t in _list[0:]]

count16= Counter( flat_tokens16)

counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
counted16_k= [k for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)]
print( len( counted16_k))
print( counted16_k)

C:/mestrado/datasets/final/datasets_pool/hand/labels/*.pgn
C:/mestrado/datasets/final/datasets_pool/torneio-except-test/labels/*.pgn
C:/mestrado/datasets/final/test-set/new-random-8lines--v1--test-only/new-random-8lines--v1--test-only/test/labels/*.pgn
2502
['C:/mestrado/datasets/final/datasets_pool/hand/labels\\a3__d5__d4__Nf6_5742_1494_R_part3_scan0145.pgn', 'C:/mestrado/datasets/final/datasets_pool/hand/labels\\a3__d5__d4__Nf6_5742_2491_L_part3_scan0644.pgn']
237
['O-O', 'Nf6', 'Nf3', 'd4', 'Nc3', 'e4', 'c5', 'Nc6', 'e6', 'd5', 'd6', 'c4', 'e5', 'g6', 'Bg7', 'cxd4', 'Be7', 'g3', 'Bg2', 'Nxd4', 'a6', 'e3', 'c6', 'Bd3', 'Be2', 'd3', 'c3', 'Bg5', 'Nbd7', 'Be3', 'exd5', 'cxd5', 'b6', 'Bc4', 'Bb7', 'Nbd2', 'b3', 'h6', 'b5', 'Bb4', 'h3', 'Bb5', 'Qc7', 'f4', 'Re1', 'Bb2', 'Qd2', 'a3', 'Bb3', 'Bg4', 'Qc2', 'Bd6', 'Bf4', 'Bd7', 'exd4', 'dxc4', 'Nge2', 'Nge7', 'a4', 'f3', 'Qb6', 'Nxd5', 'Bf5', 'Bc5', 'Nd2', 'Be6', 'Nd7', 'Nb3', 'Ba4', 'f5', 'dxe4', 'dxc5', 'a5', 'Nxe4', 'Bxf6', 'bxc6', 'Ne7',

In [116]:
lance_1 = counted16_k    #todos

In [119]:
lance_2 = counted16_k   # manual + teste

In [128]:
','.join( sorted( lance_1))

'Ba4,Ba5,Ba6,Ba7,Bb2,Bb3,Bb4,Bb4+,Bb5,Bb5+,Bb6,Bb7,Bc2,Bc4,Bc5,Bd2,Bd3,Bd6,Bd7,Be2,Be3,Be4,Be5,Be6,Be7,Bf3,Bf4,Bf5,Bg2,Bg3,Bg4,Bg5,Bg6,Bg7,Bh3,Bh4,Bh5,Bh6,Bh7,Bxb4,Bxb7,Bxc3,Bxc3+,Bxc4,Bxc5,Bxc6,Bxc6+,Bxc7,Bxd2+,Bxd3,Bxd4,Bxd6,Bxd7+,Bxe4,Bxf3,Bxf4,Bxf5,Bxf6,Bxg2,Bxg7,Kd8,Kxd8,Kxg7,N1c3,Na3,Na4,Na5,Na6,Nb3,Nb4,Nb5,Nb6,Nbd2,Nbd7,Nbxd2,Nc2,Nc3,Nc4,Nc5,Nc6,Nc7,Nce2,Nce7,Nd2,Nd4,Nd5,Nd6+,Nd7,Ndb5,Nde2,Ne1,Ne2,Ne4,Ne5,Ne6,Ne7,Nf1,Nf3,Nf5,Nf6,Nf7,Nfd7,Ng4,Ng5,Ng6,Nge2,Nge7,Ngf3,Ngf6,Nh4,Nh5,Nxc3,Nxc4,Nxc6,Nxd4,Nxd5,Nxd7,Nxe4,Nxe5,Nxf4,Nxf6,Nxf6+,Nxg2,Nxg3,O-O,O-O-O,Qa4,Qa4+,Qa5,Qa5+,Qb3,Qb6,Qc1,Qc2,Qc3,Qc5,Qc7,Qd1,Qd2,Qd3,Qd4,Qd5,Qd6,Qd7,Qe1,Qe2,Qe3,Qe7,Qe8,Qf3,Qf6,Qg4,Qg5,Qg6,Qh4,Qh5+,Qxb2,Qxb3,Qxb6,Qxb7,Qxc3,Qxc4,Qxc7,Qxd3,Qxd4,Qxd5,Qxd6,Qxd7,Qxd8+,Qxe4+,Qxf3,Qxf6,Qxg2,Ra7,Rb1,Rb8,Rc8,Re1,Re1+,Re8,a3,a4,a5,a6,axb6,b3,b4,b5,b6,bxc3,bxc6,c3,c4,c5,c6,cxb3,cxb4,cxd4,cxd5,cxd6,d3,d4,d5,d6,dxc3,dxc4,dxc5,dxc6,dxe4,dxe5,e3,e4,e5,e6,exd4,exd5,exd6,exf4,exf5,exf6,f3,f4,f5,f6,fxe4,fxe5,g3,g4,g5,g6,gx

In [129]:
','.join( sorted( lance_2))

'Ba4,Ba5,Ba7,Bb2,Bb3,Bb4,Bb4+,Bb5,Bb5+,Bb7,Bc4,Bc5,Bd2,Bd3,Bd6,Bd7,Be2,Be3,Be5,Be6,Be7,Bf3,Bf4,Bf5,Bg2,Bg3,Bg4,Bg5,Bg7,Bh4,Bh6,Bh7,Bxb4,Bxc3,Bxc3+,Bxc4,Bxc5,Bxc6,Bxc7,Bxd2+,Bxd6,Bxd7+,Bxf6,Kd8,Kxd8,Na4,Na5,Na6,Nb3,Nb4,Nb5,Nb6,Nbd2,Nbd7,Nbxd2,Nc2,Nc3,Nc5,Nc6,Nce7,Nd2,Nd4,Nd5,Nd6+,Nd7,Ndb5,Nde2,Ne1,Ne2,Ne4,Ne5,Ne7,Nf1,Nf3,Nf6,Nfd7,Ng5,Nge2,Nge7,Ngf3,Ngf6,Nh5,Nxc6,Nxd4,Nxd5,Nxd7,Nxe4,Nxe5,Nxf6+,O-O,O-O-O,Qa4,Qa5,Qa5+,Qb3,Qb6,Qc2,Qc3,Qc7,Qd1,Qd2,Qd3,Qd4,Qd5,Qd6,Qd7,Qe1,Qe2,Qe7,Qe8,Qf3,Qf6,Qg6,Qh4,Qxb2,Qxb3,Qxc4,Qxc7,Qxd4,Qxd5,Qxd6,Qxd8+,Qxe4+,Qxf6,Rb1,Rb8,Rc8,Re1,Re8,a3,a4,a5,a6,b3,b4,b5,b6,bxc3,bxc6,c3,c4,c5,c6,cxb3,cxd4,cxd5,cxd6,d3,d4,d5,d6,dxc3,dxc4,dxc5,dxc6,dxe4,e3,e4,e5,e6,exd4,exd5,exf6,f3,f4,f5,f6,g3,g5,g6,gxf6,h3,h4,h5,h6'

In [130]:
','.join( sorted( [w for w in lance_1 if w not in lance_2]))

'Ba6,Bb6,Bc2,Be4,Bg6,Bh3,Bh5,Bxb7,Bxc6+,Bxd3,Bxd4,Bxe4,Bxf3,Bxf4,Bxf5,Bxg2,Bxg7,Kxg7,N1c3,Na3,Nc4,Nc7,Nce2,Ne6,Nf5,Nf7,Ng4,Ng6,Nh4,Nxc3,Nxc4,Nxf4,Nxf6,Nxg2,Nxg3,Qa4+,Qc1,Qc5,Qe3,Qg4,Qg5,Qh5+,Qxb6,Qxb7,Qxc3,Qxd3,Qxd7,Qxf3,Qxg2,Ra7,Re1+,axb6,cxb4,dxe5,exd6,exf4,exf5,fxe4,fxe5,g4,gxf5,hxg3'

In [131]:
len( [w for w in lance_1 if w not in lance_2])

62

In [121]:
[w for w in lance_2 if w not in lance_1]

[]

In [ ]:
C:\mestrado\datasets\final\datasets_pool\torneio-all\labels\*.pgn

In [75]:
files_test_all=get_files('C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/torneio-except-test/labels/*.pgn')
files_test_except=('C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/torneio-all/labels/*.pgn')
files_test= ('C:/mestrado/datasets/final/test-set/new-random-8lines--v1--test-only/new-random-8lines--v1--test-only/test/labels/*.pgn')

In [76]:
def get_files( path):
    files = glob( path)
    files = [Path( f).name for f in files]
    return files

get_files('C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/torneio-except-test/labels/*.pgn') 


['carnaval_0001_line_1_8.pgn',
 'carnaval_0002_line_1_8.pgn',
 'carnaval_0003_line_1_8.pgn',
 'carnaval_0005_line_1_8.pgn',
 'carnaval_0007_line_1_8.pgn',
 'carnaval_0013_line_1_8.pgn',
 'carnaval_0016_line_1_8.pgn',
 'carnaval_0017_line_1_8.pgn',
 'carnaval_0020_line_1_8.pgn',
 'carnaval_0021_line_1_8.pgn',
 'carnaval_0024_line_1_8.pgn',
 'carnaval_0025_line_1_8.pgn',
 'carnaval_0026_line_1_8.pgn',
 'carnaval_0027_line_1_8.pgn',
 'carnaval_0030_line_1_8.pgn',
 'carnaval_0035_line_1_8.pgn',
 'carnaval_0037_line_1_8.pgn',
 'carnaval_0039_line_1_8.pgn',
 'carnaval_0041_line_1_8.pgn',
 'carnaval_0043_line_1_8.pgn',
 'carnaval_0044_line_1_8.pgn',
 'carnaval_0045_line_1_8.pgn',
 'carnaval_0046_line_1_8.pgn',
 'carnaval_0047_line_1_8.pgn',
 'carnaval_0048_line_1_8.pgn',
 'carnaval_0049_line_1_8.pgn',
 'carnaval_0050_line_1_8.pgn',
 'carnaval_0052_line_1_8.pgn',
 'carnaval_0053_line_1_8.pgn',
 'carnaval_0054_line_1_8.pgn',
 'carnaval_0057_line_1_8.pgn',
 'carnaval_0060_line_1_8.pgn',
 'carnav

In [102]:
files_test_except=get_files('C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/torneio-except-test/labels/*.pgn')
files_test_all=get_files('C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/torneio-all/labels/*.pgn')
files_test= get_files('C:/mestrado/datasets/final/test-set/new-random-8lines--v1--test-only/new-random-8lines--v1--test-only/test/labels/*.pgn')

In [106]:
print( len( files_test_except))
print( len( files_test_all))
print( len( files_test))

378
471
114


In [107]:
[f for f in files_test if f in files_test_except]

[]

In [108]:
[f for f in files_test_except if f in files_test]

[]

In [109]:
len( [f for f in files_test if f not in files_test_all])

114

In [110]:
len( [f for f in files_test_except if f not in files_test_all])

0

In [111]:
len( [f for f in files_test_all if f in files_test])

0

In [ ]:
len( [f for f in files_test_all if f in files_test])